In [54]:
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

%matplotlib inline

In [55]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [56]:
maxLen = len(max(X_train, key=len).split())

In [57]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [58]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [59]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m , max_len))
    
    for i in range(m):                              # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        words = X[i].lower().split()
        
        # Initialize j to 0
        j =0
        
        for j in range(len(words)):# Loop over the words of sentence_words
      
            X_indices[i][j] = word_to_index[words[j]] # Set the (i,j)th entry of X_indices to the index of the correct word.
           
            # Increment j to j + 1
            
            
    ### END CODE HERE ###
    
    return X_indices

In [60]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Step 1
    # Initialize the embedding matrix as a numpy array of zeros.
    # See instructions above to choose the correct shape.
    emb_matrix = np.zeros((vocab_len , emb_dim ))
    
    # Step 2
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    
    for key ,idx in word_to_index.items():
        emb_matrix[idx] =  word_to_vec_map[key]

    # Step 3
    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)

    ### END CODE HERE ###

    # Step 4 (already done for you; please do not modify)
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [61]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph
    # It should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = (input_shape) , dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    emb_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = emb_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    lstm = LSTM(units = 128 , return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    lstm_dropout = Dropout(rate = 0.5)(lstm)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    lstm = LSTM(units = 128, return_sequences = False)(lstm_dropout)
    # Add dropout with a probability of 0.5
    lstm_dropout = Dropout(rate = 0.5)(lstm)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    lstm_softmax = Dense(units = 5)(lstm_dropout)
    # Add a softmax activation
    softmax_activation = Activation('softmax')(lstm_softmax)
    
    # Create Model instance which converts sentence_indices into X.
   
    model = Model(inputs = sentence_indices , outputs = softmax_activation )
    ### END CODE HERE ###
    
    return model

In [62]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645 

In [63]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
print(Y_train.shape)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)
print(X_train_indices.shape)
print(Y_train_oh.shape)

(132,)
(132, 10)
(132, 5)


In [65]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 1s 5ms/step - loss: 1.5973 - accuracy: 0.1818
Epoch 2/50
132/132 [==============================] - 0s 760us/step - loss: 1.5175 - accuracy: 0.2879
Epoch 3/50
132/132 [==============================] - 0s 795us/step - loss: 1.4589 - accuracy: 0.3106
Epoch 4/50
132/132 [==============================] - 0s 792us/step - loss: 1.3854 - accuracy: 0.4318
Epoch 5/50
132/132 [==============================] - 0s 855us/step - loss: 1.2776 - accuracy: 0.5682
Epoch 6/50
132/132 [==============================] - 0s 819us/step - loss: 1.1359 - accuracy: 0.6439
Epoch 7/50
132/132 [==============================] - 0s 780us/step - loss: 0.9773 - accuracy: 0.6818
Epoch 8/50
132/132 [==============================] - 0s 811us/step - loss: 0.8560 - accuracy: 0.6742
Epoch 9/50
132/132 [==============================] - 0s 793us/step - loss: 0.8539 - accuracy: 0.6894
Epoch 10/50
132/132 [==============================] - 0s 774us/step - loss: 0.7743 

In [66]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 3ms/step

Test accuracy =  0.7678571343421936
